<h4>(1 - Recommend). Create and save new notebook(s) to enable automatic re-mounting of Drive storage. Copy and paste the below code into a <a href="http://colab.research.google.com/#create=true" target="_parent">new notebook</a> in Colab and follow the directions in the <a href="../#how-to-use" target="_parent">How To Use</a> section of the <em>README</em></h4><h4>(2). Open the pre-formed version in Colab (requires manual authorization each time a notebook is opened) <a href="https://colab.research.google.com/github/tdulcet/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabCPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4>

In [ ]:
#@title <h1><strong>📔 Colab CPU Notebook</strong></h1>#{ output-height: 8, form-width: "96%", display-mode: "form" }

#@markdown <h4>↖️ Click the ▶ button after deciding on the options below.</h4>
#@markdown <h4>💡 Keep each notebook <strong>open</strong> in the browser or 📌 <a href="https://support.mozilla.org/en-US/kb/pinned-tabs-keep-favorite-websites-open" target="_blank">the tab</a> to a dedicated window to prevent disconnection.</h4>
#@markdown <h4>🐛 The <em>debug</em> field outputs CPU (Prime95) progress, then exits.</h4>
#@markdown <h4>📜 Please see the <a href="https://github.com/tdulcet/Distributed-Computing-Scripts/tree/master/google-colab" target="_blank">documentation</a> for more information.</h4>
#@markdown <h4>🤷 Optionally, create a GIMPS/PrimeNet account <a href="https://www.mersenne.org/update/" target="_blank">here</a> and <a href="https://www.mersenne.org/jteam/" target="_blank">join</a> the “Portland State University” team!</h4>

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
type_of_work = '100 - First time LL tests' #@param ['0 - Whatever makes the most sense', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '153 - 100 million digit PRP tests', '160 - First time PRP on Mersenne cofactors', '161 - Double-check PRP on Mersenne cofactors' ]
computer_number = "Default (1)" #@param ["Default (1)", "2"]{allow-input: true}
local_time = 'Pacific' #@param ['Pacific', 'Mountain', 'Central', 'Eastern', 'Alaska', 'Hawaii']
debug = 'False' #@param ['False', 'CPU (Prime95)']

import os

class StopExecution(Exception):
  def _render_traceback_(self):
      pass

def load_drive():
  '''Load & cd into gdrive for persistent data'''
  if os.path.exists("/content/gdrive/My Drive"): # use our notebook
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd "/content/gdrive/My Drive/"
  elif os.path.exists("/content/drive/My Drive"): # create your own notebook with our code
    %cd "/content/drive/My Drive"
  if "My Drive/GIMPS" in os.getcwd(): # don't create a subfolder in GIMPS/
    return
  !mkdir -p GIMPS
  %cd "GIMPS"

def run():
  '''Run MPrime'''
  !cd mprime_cpu && chmod 777 mprime && ./mprime -A$computer_number -d | tee -a "{'cpu' + computer_number + '.out'}"

def debug_exit():
  '''Output of Prime95 CPU outputs'''
  if debug == 'CPU (Prime95)' and os.path.exists('mprime_cpu/cpu' + computer_number + '.out'):
    print("\nCPU (Prime95) output:\n")
    !tail -n 100 "{'mprime_cpu/cpu' + computer_number + '.out'}" # view MPrime progress
    !cd mprime_cpu && chmod 777 mprime && echo -e "3\n\n5" | ./mprime -m
    print()
  else:
    print(f'No `{debug}` output file found for debug option and computer number `{computer_number}`.\n')

def install():
  '''Download/Install/Configure Prime95'''
  !wget https://github.com/tdulcet/Distributed-Computing-Scripts/archive/master.zip -nv -O master.zip
  !unzip -o master.zip
  !ln -s Distributed-Computing-Scripts-master/{mprime2.sh,mprime2.exp,idletime.sh} . # -- TODO -- test
  !sed -i 's/"mprime"/"mprime_cpu"/' Distributed-Computing-Scripts-master/mprime2.sh # Name the folder specific to the runtime type
  !sed -i '/^nohup / s/^/# /' Distributed-Computing-Scripts-master/mprime2.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' Distributed-Computing-Scripts-master/mprime2.sh # Do not create a cronjob
  !sed -i '/^expect {/a \\t"Upload bandwidth limit in Mbps (*):" { sleep 1; send -- "1000\\r"; exp_continue }\n\t"Skip advanced resource settings (*):" { sleep 1; send -- "n\\r"; exp_continue }\n\t"Optional directory to hold *:" { sleep 1; send -- "\\r"; exp_continue }\n\t"stage 2 memory in GB (*):" { sleep 1; send -- "6\\r"; exp_continue }\n\t"Max emergency memory in GB/worker (*):" { sleep 1; send -- "6\\r"; exp_continue }' Distributed-Computing-Scripts-master/mprime2.exp
  #!bash -- Distributed-Computing-Scripts-master/mprime2.sh $computer_number $prime_ID $computer_name $type_of_work # Run script
  !bash -- mprime_cpu/mprime2.sh $computer_number $prime_ID $computer_name $type_of_work # Run script -- TODO -- test
  !echo 'FixedHardwareUID=1' >> "{'prim000' + computer_number + '.txt'}"
  run()

load_drive()

# set local time
!rm -f /etc/localtime
!ln -s {'/usr/share/zoneinfo/US/' + local_time} /etc/localtime

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computers_name = "" if computer_name.lower() == 'default' else computer_name
type_of_work = type_of_work.split("-")[0].rstrip()
computer_number = "1" if computer_number.lower() == 'default (1)' else computer_number
debug = False if debug == 'False' else debug

# Add quotes to string args so script can parse spaces/special characters
prime_ID = f'"'+ prime_ID + f'"'
computer_name = f'"'+ computer_name + f'"'

!wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info

if debug:
  debug_exit()
  raise StopExecution

elif not computer_number.isdigit() or int(computer_number) < 0 or int(computer_number) > 9:
  print("ERROR: Computer number must be a number from 0-9")
  raise StopExecution

elif os.path.exists('mprime_cpu/work000' + computer_number + '.txt'):
  run()

else:
  install()

print("Gracefully exiting...")